![Credit card being held in hand](credit_card.jpg)

Commercial banks receive _a lot_ of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. Manually analyzing these applications is mundane, error-prone, and time-consuming (and time is money!). 

### The Data

The data is a small subset of the Credit Card Approval dataset from the UCI Machine Learning Repository showing the credit card applications a bank receives. The last column in the dataset is the target value.

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')


# Load the dataset
cc_apps = pd.read_csv("cc_approvals.data", header=None) 
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,g,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,g,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,g,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,g,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,s,0,+


In [2]:
#convert column names as str
cc_apps.columns = cc_apps.columns.astype(str)

In [3]:
cc_apps.shape

(690, 14)

In [4]:
cc_apps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       690 non-null    object 
 1   1       690 non-null    object 
 2   2       690 non-null    float64
 3   3       690 non-null    object 
 4   4       690 non-null    object 
 5   5       690 non-null    object 
 6   6       690 non-null    object 
 7   7       690 non-null    float64
 8   8       690 non-null    object 
 9   9       690 non-null    object 
 10  10      690 non-null    int64  
 11  11      690 non-null    object 
 12  12      690 non-null    int64  
 13  13      690 non-null    object 
dtypes: float64(2), int64(2), object(10)
memory usage: 75.6+ KB


In [5]:
cc_apps.duplicated().sum()

0

In [6]:
cc_apps.describe()

,2,7,10,12
count,690.000000,690.000000,690.00000,690.000000
mean,4.758725,2.223406,2.40000,1017.385507
std,4.978163,3.346513,4.86294,5210.102598
min,0.000000,0.000000,0.00000,0.000000
25%,1.000000,0.165000,0.00000,0.000000
50%,2.750000,1.000000,0.00000,5.000000
75%,7.207500,2.625000,3.00000,395.500000
max,28.000000,28.500000,67.00000,100000.000000


In [7]:
# Remove '?' 
cc_apps = cc_apps.replace(r'\?', np.NaN, regex=True)

In [8]:
cc_apps= cc_apps.dropna()
cc_apps.shape

(659, 14)

In [9]:
cc_apps.isna().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
dtype: int64

In [10]:
# Convert column 1 to float
cc_apps['1'] = cc_apps['1'].astype(float)

In [11]:
# Model Dev
X = cc_apps.drop(['13'], axis=1)
y = cc_apps['13']

In [12]:
X = pd.get_dummies(X, drop_first = True)
X.head()

,1,2,7,10,12,0_b,3_u,3_y,4_gg,4_p,...,6_h,6_j,6_n,6_o,6_v,6_z,8_t,9_t,11_p,11_s
0,30.83,0.000,1.25,1,0,1,1,0,0,0,...,0,0,0,0,1,0,1,1,0,0
1,58.67,4.460,3.04,6,560,0,1,0,0,0,...,1,0,0,0,0,0,1,1,0,0
2,24.50,0.500,1.50,0,824,0,1,0,0,0,...,1,0,0,0,0,0,1,0,0,0
3,27.83,1.540,3.75,5,3,1,1,0,0,0,...,0,0,0,0,1,0,1,1,0,0
4,20.17,5.625,1.71,0,0,1,1,0,0,0,...,0,0,0,0,1,0,1,0,0,1


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 3, stratify = y)
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

lr = LogisticRegression()

param_grid = {
    'C' : [0.01, 0.1, 1, 10, 100],
    'solver' : ['liblinear', 'lbfgs'],
    'penalty' : ['l1', 'l2']
}

gc = GridSearchCV(estimator = lr,
                 param_grid = param_grid,
                 cv = 5,
                 scoring = 'accuracy',
                 verbose = 1)

gc.fit(X_train_scaled, y_train)

best_score = gc.best_score_
print(best_score)

y_pred = gc.predict(X_test_scaled)

cm = confusion_matrix(y_test, y_pred)
print(cm)

acs =accuracy_score(y_test, y_pred)
print(acs)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
0.8704596990311275
[[71  4]
 [22 68]]
0.8424242424242424
